# Julia 機器學習：DecisionTree 決策樹

## 作業 030：乳癌預測資料集

請使用隨機森林模型建立一個分類模型，來預測乳癌資料集中，為良性或是惡性的腫瘤。

In [1]:
using DecisionTree, RDatasets, DataFrames, MLDataUtils, Statistics

┌ Info: Recompiling stale cache file /Users/prestonyu9011/.julia/compiled/v1.2/RDatasets/JyIbx.ji for RDatasets [ce6b1742-4840-55fa-b093-852dadbb1d8b]
└ @ Base loading.jl:1240
┌ Info: Recompiling stale cache file /Users/prestonyu9011/.julia/compiled/v1.2/MLDataUtils/CQWB9.ji for MLDataUtils [cc2ba9b6-d476-5e6d-8eaf-a92d5412d41d]
└ @ Base loading.jl:1240


## 讀取資料

In [2]:
biopsy = dataset("MASS", "biopsy")

,ID,V1,V2,V3,V4,V5,V6,V7,V8,V9,Class
,String,Int32,Int32,Int32,Int32,Int32,Int32?,Int32,Int32,Int32,Cat…
1,1000025,5,1,1,1,2,1,3,1,1,benign
2,1002945,5,4,4,5,7,10,3,2,1,benign
3,1015425,3,1,1,1,2,2,3,1,1,benign
4,1016277,6,8,8,1,3,4,3,7,1,benign
5,1017023,4,1,1,3,2,1,3,1,1,benign
6,1017122,8,10,10,8,7,10,9,7,1,malignant
7,1018099,1,1,1,1,2,10,3,1,1,benign
8,1018561,2,1,2,1,2,1,3,1,1,benign
9,1033078,2,1,1,1,2,1,1,1,5,benign


In [3]:
describe(biopsy)

,variable,mean,min,median,max,nunique,nmissing,eltype
,Symbol,Union…,Any,Union…,Any,Union…,Union…,Type
1,ID,,1000025,,95719,645,,String
2,V1,4.41774,1,4.0,10,,,Int32
3,V2,3.13448,1,1.0,10,,,Int32
4,V3,3.20744,1,1.0,10,,,Int32
5,V4,2.80687,1,1.0,10,,,Int32
6,V5,3.21602,1,2.0,10,,,Int32
7,V6,3.54466,1,1.0,10,,16,"Union{Missing, Int32}"
8,V7,3.43777,1,3.0,10,,,Int32
9,V8,2.86695,1,1.0,10,,,Int32


### Train, Test Split

In [4]:
train_features = dropmissing(biopsy)[1:2:end, 2:10];
train_labels = dropmissing(biopsy)[1:2:end, 11];
test_features = dropmissing(biopsy)[2:2:end, 2:10];
test_labels = dropmissing(biopsy)[2:2:end, 11];

In [5]:
train_features = convert(Array{Int}, train_features);
train_labels = convert(Array{String}, train_labels);
test_features = convert(Array{Int}, test_features);
test_labels = convert(Array{String}, test_labels);

### Random Forest

In [6]:
model = DecisionTree.RandomForestClassifier(n_trees=30, max_depth=2)

RandomForestClassifier
n_trees:             30
n_subfeatures:       -1
partial_sampling:    0.7
max_depth:           2
min_samples_leaf:    1
min_samples_split:   2
min_purity_increase: 0.0
classes:             nothing
ensemble:            nothing

In [7]:
# Training
DecisionTree.fit!(model, train_features, train_labels)

RandomForestClassifier
n_trees:             30
n_subfeatures:       -1
partial_sampling:    0.7
max_depth:           2
min_samples_leaf:    1
min_samples_split:   2
min_purity_increase: 0.0
classes:             ["benign", "malignant"]
ensemble:            Ensemble of Decision Trees
Trees:      30
Avg Leaves: 3.8666666666666667
Avg Depth:  2.0

In [8]:
# Prediction
predicted_labels = DecisionTree.predict(model, test_features);

In [9]:
# Evaluation
using Printf
@printf("Accuracy: %.2f%%\n", mean((predicted_labels .== test_labels))*100)

Accuracy: 97.65%
